In [2]:
conda install -c anaconda py-xgboost

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [3]:
from xgboost import XGBClassifier

In [4]:
import pandas as pd
import numpy as np
from sklearn import datasets

In [5]:
iris = datasets.load_iris()

In [9]:
df = pd.DataFrame(data = np.c_[iris['data'], iris['target']], columns= [iris.feature_names + ['target']])

In [10]:
df.head(2)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0


In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X_train, X_test, y_train, y_test = train_test_split(iris['data'], iris['target'], random_state=2)

In [13]:
from sklearn.metrics import accuracy_score

In [16]:
#gbtree == gradient boosting tree?, softmax, 트리 깊이를 6개, 트리의 개수 100개, cpu 전체 다 쓸께
xgb = XGBClassifier(booster = 'gbtree', objective= 'multi:softprob', max_depth = 6,
              learning_rate = 0.1, n_estimators = 100, n_jobs = -1)

In [18]:
xgb.fit(X_train, y_train)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=-1, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [19]:
xgb.predict(X_test)

array([0, 0, 2, 0, 0, 2, 0, 2, 2, 0, 0, 0, 0, 0, 1, 1, 0, 1, 2, 1, 2, 1,
       2, 1, 1, 0, 0, 2, 0, 2, 2, 0, 1, 2, 1, 0, 2, 1], dtype=int64)

In [20]:
y_test

array([0, 0, 2, 0, 0, 2, 0, 2, 2, 0, 0, 0, 0, 0, 1, 1, 0, 1, 2, 1, 1, 1,
       2, 1, 1, 0, 0, 2, 0, 2, 2, 0, 1, 2, 1, 0, 2, 1])

In [21]:
accuracy_score(xgb.predict(X_test), y_test)

0.9736842105263158

In [22]:
X,y = datasets.load_diabetes(return_X_y=True)

In [23]:
X.shape

(442, 10)

In [25]:
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor

In [26]:
xgb_reg = XGBRegressor(booster='gbtree', objective='reg:squarederror', max_depth=6, 
             learning_rate = 0.1, n_estimators=100, n_jobs=-1)


In [28]:
scores = cross_val_score(xgb_reg, X, y, scoring= 'neg_mean_squared_error', cv = 5)

In [31]:
rmse = np.sqrt(-scores) # root m?? squared error

In [32]:
rmse.mean()

63.12404095304173

In [34]:
pd.DataFrame(y).describe() # 기술통계값??

,0
count,442.000000
mean,152.133484
std,77.093005
min,25.000000
25%,87.000000
50%,140.500000
75%,211.500000
max,346.000000


In [44]:
df = pd.read_csv("./source/atlas-higgs-challenge-2014-v2.csv.gz")

In [45]:
df.head(3)

,EventId,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,DER_pt_tot,DER_sum_pt,DER_pt_ratio_lep_tau,DER_met_phi_centrality,DER_lep_eta_centrality,PRI_tau_pt,PRI_tau_eta,PRI_tau_phi,PRI_lep_pt,PRI_lep_eta,PRI_lep_phi,PRI_met,PRI_met_phi,PRI_met_sumet,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt,Weight,Label,KaggleSet,KaggleWeight
0,100000,138.470,51.655,97.827,27.980,0.91,124.711,2.666,3.064,41.928,197.760,1.582,1.396,0.2,32.638,1.017,0.381,51.626,2.273,-2.414,16.824,-0.277,258.733,2,67.435,2.150,0.444,46.062,1.24,-2.475,113.497,0.000814,s,t,0.002653
1,100001,160.937,68.768,103.235,48.146,-999.00,-999.000,-999.000,3.473,2.078,125.157,0.879,1.414,-999.0,42.014,2.039,-3.011,36.918,0.501,0.103,44.704,-1.916,164.546,1,46.226,0.725,1.158,-999.000,-999.00,-999.000,46.226,0.681042,b,t,2.233584
2,100002,-999.000,162.172,125.953,35.635,-999.00,-999.000,-999.000,3.148,9.336,197.814,3.776,1.414,-999.0,32.154,-0.705,-2.093,121.409,-0.953,1.052,54.283,-2.186,260.414,1,44.251,2.053,-2.028,-999.000,-999.00,-999.000,44.251,0.715742,b,t,2.347389


In [46]:
df.shape

(818238, 35)

In [47]:
label_col = df['Label']

In [49]:
df.drop(['Weight', 'KaggleSet', 'Label'], axis=1, inplace= True)

In [51]:
df.rename(columns={'KaggleWeight' : 'Weight'}, inplace= True)

In [52]:
df.head(3)

,EventId,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,DER_pt_tot,DER_sum_pt,DER_pt_ratio_lep_tau,DER_met_phi_centrality,DER_lep_eta_centrality,PRI_tau_pt,PRI_tau_eta,PRI_tau_phi,PRI_lep_pt,PRI_lep_eta,PRI_lep_phi,PRI_met,PRI_met_phi,PRI_met_sumet,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt,Weight
0,100000,138.470,51.655,97.827,27.980,0.91,124.711,2.666,3.064,41.928,197.760,1.582,1.396,0.2,32.638,1.017,0.381,51.626,2.273,-2.414,16.824,-0.277,258.733,2,67.435,2.150,0.444,46.062,1.24,-2.475,113.497,0.002653
1,100001,160.937,68.768,103.235,48.146,-999.00,-999.000,-999.000,3.473,2.078,125.157,0.879,1.414,-999.0,42.014,2.039,-3.011,36.918,0.501,0.103,44.704,-1.916,164.546,1,46.226,0.725,1.158,-999.000,-999.00,-999.000,46.226,2.233584
2,100002,-999.000,162.172,125.953,35.635,-999.00,-999.000,-999.000,3.148,9.336,197.814,3.776,1.414,-999.0,32.154,-0.705,-2.093,121.409,-0.953,1.052,54.283,-2.186,260.414,1,44.251,2.053,-2.028,-999.000,-999.00,-999.000,44.251,2.347389


In [54]:
# s > signal(신호), b > background(잡음)
label_col.replace(('s', 'b'), (1,0), inplace= True)

In [56]:
X = df.iloc[ : , 1 : ]

In [57]:
y = label_col

In [58]:
y.value_counts()

0    538678
1    279560
Name: Label, dtype: int64

In [59]:
import xgboost as xgb

In [65]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [66]:
y_train.value_counts()

0    430895
1    223695
Name: Label, dtype: int64

In [67]:
y_test.value_counts()

0    107783
1     55865
Name: Label, dtype: int64

In [68]:
xgbmat = xgb.DMatrix(X_train, y_train, missing= 999.0)

In [70]:
param = {
    'objective' : 'binary:logitraw',
    'eta' : 0.1, # 합승률?
    'max_depth' : 6,
    'eval_metric' : 'auc'
}

In [71]:
 # 부팅값을 120으로, xgbmat안에 train
bst = xgb.train(param, xgbmat, 120, [(xgbmat, 'train')])

c:\Users\Playdata\Anaconda3\lib\site-packages\xgboost\core.py:617: FutureWarning: Pass `evals` as keyword args.
  warnings.warn(msg, FutureWarning)


[0]	train-auc:1.00000
[1]	train-auc:1.00000
[2]	train-auc:1.00000
[3]	train-auc:1.00000
[4]	train-auc:1.00000
[5]	train-auc:1.00000
[6]	train-auc:1.00000
[7]	train-auc:1.00000
[8]	train-auc:1.00000
[9]	train-auc:1.00000
[10]	train-auc:1.00000
[11]	train-auc:1.00000
[12]	train-auc:1.00000
[13]	train-auc:1.00000
[14]	train-auc:1.00000
[15]	train-auc:1.00000
[16]	train-auc:1.00000
[17]	train-auc:1.00000
[18]	train-auc:1.00000
[19]	train-auc:1.00000
[20]	train-auc:1.00000
[21]	train-auc:1.00000
[22]	train-auc:1.00000
[23]	train-auc:1.00000
[24]	train-auc:1.00000
[25]	train-auc:1.00000
[26]	train-auc:1.00000
[27]	train-auc:1.00000
[28]	train-auc:1.00000
[29]	train-auc:1.00000
[30]	train-auc:1.00000
[31]	train-auc:1.00000
[32]	train-auc:1.00000
[33]	train-auc:1.00000
[34]	train-auc:1.00000
[35]	train-auc:1.00000
[36]	train-auc:1.00000
[37]	train-auc:1.00000
[38]	train-auc:1.00000
[39]	train-auc:1.00000
[40]	train-auc:1.00000
[41]	train-auc:1.00000
[42]	train-auc:1.00000
[43]	train-auc:1.0000

In [74]:
# xgbmat_test = xgb.DMatrix(X_test, y_test, missing= 999.0)
xgbmat_test = xgb.DMatrix(X_test, missing= 999.0)
y_hat = bst.predict(xgbmat_test)

In [75]:
y_hat

array([ 12.141323, -12.07885 , -12.07885 , ...,  12.141323, -12.07885 ,
        12.141323], dtype=float32)